<a href="https://colab.research.google.com/github/sbb2002/Portfolio/blob/main/.study/Tensorflow/ML_ch6_multinomial_n_softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# Multinomial classification

Binary classification graph가 각각 $A$, $B$, $C$가 있다고 하자.

binary classification graph를 모두 수행하면 데이터 분포가 마치 지도 위의 국가영토 안에 있는 도시같이 생겼다. \
여튼 이 graph들을 각각 단독적으로 수행하자니 일이 n배가 된다. \
그러니 이 graph를 행렬로 묶어 일을 한번에 처리할 수 있다. \
행렬로 나타내면 다음과 같다.

> $\begin{pmatrix}
w_{A1} & w_{A2} & w_{A3} \\
w_{B1} & w_{B2} & w_{B3} \\
w_{C1} & w_{C2} & w_{C3} 
\end{pmatrix}$
$\begin{pmatrix}
x_{1} \\
x_{2} \\
x_{3}
\end{pmatrix}$
$=\begin{pmatrix}
\hat{y_{A}} \\
\hat{y_{B}} \\
\hat{y_{C}}
\end{pmatrix}$
; A, B, C 는 label로, 3개의 binary classification graph

전에는 $H(x)$를 sigmoid로 적용함으로써 각 graph의 예측값 $\hat{y}$를 얻을 수 있었다. 그러나 기존의 예측값은 절대치여서 정수화된 데이터에서는 적합하지 않았고, 각 graph의 예측에서 실제값 $y$의 등장확률을 알 수는 없었다. \
남자=0, 여자=1인데 어떤 사람에 대한 예측값이 $\hat{y}=0.3$ 이면 이 얼마나 황당한가;;

그래서 이번에는 정수화된 데이터에 맞춰 제대로 예측해보자. \
예측값 $\hat{y}$의 등장확률을 연산하기 위해 softmax를, 그리고 $\hat{y}$를 점지(?)하기위해 one-hot encoding를 사용한다.

---

**가령 예를 들어,** \
sigmoid만 사용하여 hypothesis를 사용하여 예측한 결과
> Score: [[20.0], [5.0], [1.0]]

softmax와 one-hot encoding을 사용하여 예측한 결과
> Probability: [[0.7], [0.2], [0.1]] \
 해석: 20점 등장확률=70%, 5점 등장확률=20%, 1점 등장확률=10%

> One-Hot Encoding: [[1.0], [0.0], [0.0]] \
해석: 20점이 나올 확률이 가장 높다!

---

## Cross-entropy cost function

Multinomial classification에서의 cost함수는 아래와 같다.

> $ \begin{align*}
D(S,L) &= - \sum_{i} L_{i} \circ log(S_{i}) & \\
; & S(y) = 예측값(probability) & \text{<- 기존 가설 } H(x) \\
; & L = 실제값 & \text{<- 기존 실제값 } y &
\end{align*} $



왜 이런 cost를 쓰게 되었을까? 괄호를 다시 씌워보자. \
> $D(S,L) = \sum_{i} L_{i} \circ (-log(\hat{y_{i}})) $ \

여기서 $\circ\$는 hadamard product로 
element-wise product, point-wise product라고도 한다. \
내적과 달리 **같은 위치**의 elem끼리 곱하는 것이다.

우선 이 식의 로그꼴을 보자. \
로그 안 값이 0 이면 $\infty$, 1 이면 0이 되는 성질에 주목하면 된다. \

---

**예를 들어,** 어떤 데이터에서 다음과 같은 값을 제공했다. \
> 실제값Y $ L = \begin{pmatrix} 0 \\ 1 \end{pmatrix} $ \
옳은 예측값 $ \hat{Y} = \begin{pmatrix} 0 \\ 1 \end{pmatrix} $ , 
틀린 예측값 $ \hat{Y} = \begin{pmatrix} 1 \\ 0 \end{pmatrix} $ 

cost함수에 이 값들을 대입해보자. \


> $ \begin{align*}
D(S,L) &= \begin{pmatrix} 0 \\ 1 \end{pmatrix} \circ (-log(\begin{pmatrix} 0 \\ 1 \end{pmatrix})) \\
&= \begin{pmatrix} 0 \\ 1 \end{pmatrix} \circ \begin{pmatrix} \infty \\ 0 \end{pmatrix} \\
&= \begin{pmatrix} 0 \\ 0 \end{pmatrix} = O
\end{align*} $

옳은 예측값에 대해 cost에 대해 0, 즉 최소가 되어 선택된다.

반대로 틀린 예측값을 넣으면 \
> $ \begin{align*}
D(S,L) &= \begin{pmatrix} 0 \\ 1 \end{pmatrix} \circ (-log(\begin{pmatrix} 1 \\ 0 \end{pmatrix})) \\
&= \begin{pmatrix} 0 \\ 1 \end{pmatrix} \circ \begin{pmatrix} 0 \\ \infty \end{pmatrix} \\
&= \begin{pmatrix} 0 \\ \infty \end{pmatrix}
\end{align*} $

틀린 예측값에 대한 cost는 발산(nan)하므로 적절히 걸러진다.

---

## Softmax function

아까 score(절대치)를 prob.(상대치)으로 바꿀 수 있다고 하지 않았는가? \
그 과정을 수행하는 함수가 바로 softmax function이다.

1. **hypothesis**

 score는 기존의 $H(x) = WX + b$를 가설삼아 아래 코드를 이용한다.
```
tf.matmul(X,W) + b
```
 prob.은 기존과 달리 $S(y_{i}) = \frac{e^{y_{i}}}{\sum e^{y_{i}}}$을 가설삼아 아래 코드를 이용한다.
```
tf.nn.softmax(tf.matmul(X,W) + b)
```

2. **cost(loss) function**

 $\mathcal{L} = \frac{1}{N} \sum_{i} D(S(WX_{i}+b), L_{i})$ 

 수학적으로 이렇게 정의되며, 그 코드는 아래와 같다.
```
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
```





## One-hot encoding

 숫자 0, 1, 2, 3를 원핫 인코딩으로 표현하면 각각 이렇게 표시된다.
 > [1,0,0,0]=0, [0,1,0,0]=1, [0,0,1,0]=2, [0,0,0,1]=3

 마치 자리표같은 개념이다. \
 [ ]안 0, 1은 occuipied 여부라고 생각하면 되겠다.

## Practice

In [ ]:
# Give the data
x_data = [[1,2,3,4], [2,1,3,2], [3,1,3,4], [4,1,5,5], [1,7,5,5], [1,2,5,6], [1,6,6,6], [1,7,7,7]]
y_data = [[0,0,1], [0,0,1], [0,0,1], [0,1,0], [0,1,0], [0,1,0], [1,0,0], [1,0,0]]

# Define variables
X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3      # ont-hot label의 갯수

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X,W) + b)

# Cross entropy cost
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [ ]:
# Launch graph
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for step in range(2001):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 200 == 0:
      print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
  
  print("*"*100)

  # Testing & one-hot encoding
  print("\ntest example1 )")
  a = sess.run(hypothesis, feed_dict={X: [[1,11,7,9]]})
  print("Probabilities are \t", a, "\n나올 만한 숫자는 \t", sess.run(tf.arg_max(a, 1)))

  print("\ntest example2 )")
  all = sess.run(hypothesis, feed_dict={X: [[1,11,7,9],
                                            [1,3,4,3],
                                            [1,1,0,1]]})
  print("Probabilities are \n", all, "\n나올 만한 숫자는 \t", sess.run(tf.arg_max(all, 1)))


0 3.0920093
200 0.76331615
400 0.599558
600 0.47218907
800 0.36392462
1000 0.27922708
1200 0.24726559
1400 0.22281599
1600 0.20278364
1800 0.18603793
2000 0.17181705
****************************************************************************************************

test example1 )
Probabilities are 	 [[6.1080749e-03 9.9364674e-01 2.4517259e-04]] 
나올 만한 숫자는 	 [1]

test example2 )
Probabilities are 
 [[6.1080749e-03 9.9364674e-01 2.4517259e-04]
 [7.9215646e-01 1.2212515e-01 8.5718378e-02]
 [6.7949757e-09 1.3661663e-04 9.9986339e-01]] 
나올 만한 숫자는 	 [1 0 2]


# Fancy softmax classifier

## softmax_cross_entropy_with_logits

In [ ]:
# logits = tf.matmul(X,W) + b         # scores(예측값 y) 와 동일
# hypothesis = tf.nn.softmax(logits)

In [ ]:
# Cross entropy cost
# cost = tf.reduce_mean(-tf.reduce_sum(Y_one_hot * tf.log(hypothesis), axis=1))

# Cross entropy cost that is provided by tf.nn
# cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
# cost = tf.reduce_mean(cost_i)

## tf.one_hot and reshape

### github에서 data 가져오는 방법


In [ ]:
# github에서 clone해와서 data 사용해보기
!git clone https://github.com/hunkim/DeepLearningZeroToAll/

Cloning into 'DeepLearningZeroToAll'...
remote: Enumerating objects: 1709, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 1709 (delta 2), reused 1 (delta 1), pack-reused 1697
Receiving objects: 100% (1709/1709), 751.23 KiB | 20.87 MiB/s, done.
Resolving deltas: 100% (1111/1111), done.


In [ ]:
!ls

DeepLearningZeroToAll  sample_data


In [ ]:
!ls DeepLearningZeroToAll

 chainer
 CONTRIBUTING.md
 data-01-test-score.csv
 data-02-stock_daily.csv
 data-03-diabetes.csv
 data-04-zoo.csv
 ipynb
 keras
 lab-01-basics.ipynb
 lab-02-1-linear_regression.py
 lab-02-2-linear_regression_feed.py
 lab-02-3-linear_regression_tensorflow.org.py
 lab-03-1-minimizing_cost_show_graph.py
 lab-03-2-minimizing_cost_gradient_update.py
 lab-03-3-minimizing_cost_tf_optimizer.py
 lab-03-X-minimizing_cost_tf_gradient.py
 lab-04-1-multi_variable_linear_regression.py
 lab-04-2-multi_variable_matmul_linear_regression.py
 lab-04-3-file_input_linear_regression.py
 lab-04-4-tf_reader_linear_regression.py
 lab-05-1-logistic_regression.py
 lab-05-2-logistic_regression_diabetes.py
 lab-06-1-softmax_classifier.py
 lab-06-2-softmax_zoo_classifier.py
 lab-07-1-learning_rate_and_evaluation.py
 lab-07-2-linear_regression_without_min_max.py
 lab-07-3-linear_regression_min_max.py
 lab-07-4-mnist_introduction.py
 lab-08-tensor_manipulation.ipynb
 lab-09-1-xor.py
 lab-09-2-xor-nn.py
 lab-09-3-xor-

In [ ]:
# 내가 찾는 파일경로 (출력되는 위치를 복사해 로드할 때 이용)
!find ./ -name data-04-zoo.csv

./DeepLearningZeroToAll/mxnet/data-04-zoo.csv
./DeepLearningZeroToAll/pytorch/data-04-zoo.csv
./DeepLearningZeroToAll/keras/data-04-zoo.csv
./DeepLearningZeroToAll/data-04-zoo.csv


위에서 가져온 데이터는 이렇게 이루어져있다고 한다. \
> [ : , 0:-1] = 동물들의 다리 수, 뿔이 있는지, 꼬리가 있는지 등을 feature화한 데이터, 16개 cols. \
[ : , [-1]] = 7개의 종으로 feature화한 데이터.

다행히 여기서는 알려주었지만, \
원래는 pd나 np, excel 등을 이용하여 차원정보(row, col)와 목표로 할 데이터 범위(x_data, y_data)를 숙지해야한다.

개인적으로 pd가 그래픽도 좋고 명확해서 좋긴 하지만, 연습 차 np로 분석해보자.

### Practice

In [ ]:
import numpy as np

In [ ]:
# Predicting animal type based on various features
xy = np.loadtxt('./DeepLearningZeroToAll/data-04-zoo.csv', delimiter=',', dtype=np.float32)
np.array(xy).shape        # (rows= 101, cols= 17)

(101, 17)

In [ ]:
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

nb_classes = 7      # feature 0~6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])     # 0~6, shape=[?,1] ; None or ? = 데이터 n(rows)개가 들어온다
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

Y_one_hot = tf.one_hot(Y, nb_classes)       # one hot shape=[?, 1, 7]... We don't want this.
# After being one_hot, N-rank of input indices gives (N+1)-rank of output indices. 
# It means [[0], [3]] outputs one_hot[[[1,0,0,0,0,0,0]], [0,0,0,1,0,0,0]]]
# So reshape Y_one_hot.
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])   # shape=[?, 7] ; -1 은 나머지 모두

# tf.nn.softmax computes softmax activations
# softmax = exp(logits)/ reduce_sum(exp(logits), dim)
logits = tf.matmul(X,W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# Launch graph
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  feed_dict = {X: x_data, Y: y_data}
  for step in range(2000):
    sess.run(optimizer, feed_dict=feed_dict)
    if step % 100 == 0:
      loss, acc = sess.run([cost, accuracy], feed_dict=feed_dict)
      print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))    # 포맷 문자열; 5칸여백, 소수점 3자리고정, 소수점 2자리고정 퍼센트 (기본: 소수점 내림)
  
  # Let's see if we can predict
  pred = sess.run(prediction, feed_dict={X: x_data})
  # y_data: [N, 1] = flatten => [N, ] matches pred.shape
  # example) [[1], [0]] flatten => [1, 0]
  for p, y in zip(pred, y_data.flatten()):       # zip은 각각 list의 elems들을 p, y로 넘겨주기 위함
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Loss: 5.218	Acc: 10.89%
Step:   100	Loss: 0.576	Acc: 85.15%
Step:   200	Loss: 0.389	Acc: 87.13%
Step:   300	Loss: 0.298	Acc: 92.08%
Step:   400	Loss: 0.244	Acc: 92.08%
Step:   500	Loss: 0.207	Acc: 95.05%
Step:   600	Loss: 0.179	Acc: 96.04%
Step:   700	Loss: 0.158	Acc: 96.04%
Step:   800	Loss: 0.142	Acc: 97.03%
Step:   900	Loss: 0.128	Acc: 98.02%
Step:  1000	Loss: 0.117	Acc: 98.02%
Step:  1100	Loss: 0.107	Acc: 99.01%
Step:  1200	Loss: 0.099	Acc: 99.01%
Step:  1300	Loss: 0.092	Acc: 100.00%
Step:  1400	Loss: 0.085	Acc: 100.00%
Step:  1500	Loss: 0.080	Acc: 100.00%
Step:  1600	Loss: 0.075	Acc: 100.00%
Step:  1700	Loss: 0.071	Acc: 100.00%
Step:  1800	Loss: 0.067	Acc: 100.00%
Step:  1900	Loss: 0.064	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True

Step 1300 언저리에서 acc 100%를 달성한다. 충분히 훈련이 되어 train set에 대해서는 천하무적이 된 것이다. \
유머러스하게도, test set으로 자기자신(train set)을 넣었으니 모든 데이터에 대해 완벽한 예측이 가능하다. \
그래서 [False] 하나 없이 [True]만 나오는 것이다. \

당연하게도 실제 training에서는 절대 train set과 test set을 **똑같이 주어선 안된다!**

나는 책에서 공부한 대로, data의 80%는 train set, 20%를 test set으로 떼어놓는다. \
ML_prac1에 그 코드를 활용했으니 참고하자.